Установка и импорт библиотек

In [ ]:
%pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

Основные функции препроцессинга

In [152]:
# 1. Метрика для размытия
# 10.1109/ICPR.2000.903548
def measBlur(img):
    return cv2.Laplacian(img, cv2.CV_64F).var()

# 2. Метрика для освещенности
# https://stackoverflow.com/questions/596216/formula-to-determine-perceived-brightness-of-rgb-color/56678483#56678483
def measLum(img): # plain input (in bgr)!
    # calculate luminance matrix
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255
    img_rgb[img_rgb <= 0.04045] = img_rgb[img_rgb <= 0.04045] / 12.92
    img_rgb[img_rgb > 0.04045] = pow(((img_rgb[img_rgb > 0.04045] + 0.055) / 1.055), 2.4)
    lum = img_rgb[:, :, 0] * 0.2126 + img_rgb[:, :, 1] * 0.7152 + img_rgb[:, :, 1] * 0.0722
    # take product with sobel full operator
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sob = cv2.Sobel(img_gray, -1, 1, 1, ksize = 7) / 255 # ksize value!
    return np.mean(lum * sob)
# 3. Метрика для шумов
#
def measNoise(img):
    return 0